In [1]:
import torch
from torchvision.io import read_image
from torchvision import transforms
from PIL import Image                                 
import torchvision.transforms.functional as TF       
import numpy as np
import matplotlib.pyplot as plt
import os
from os import listdir

In [2]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/0_goose.jpg     
  inflating: dataset/10_airliner.jpg  
  inflating: dataset/11_green mamba.jpg  
  inflating: dataset/12_green mamba.jpg  
  inflating: dataset/13_green mamba.jpg  
  inflating: dataset/14_green mamba.jpg  
  inflating: dataset/15_green mamba.jpg  
  inflating: dataset/16_goose.jpg    
  inflating: dataset/17_goose.jpg    
  inflating: dataset/18_goose.jpg    
  inflating: dataset/19_Japanese spaniel.jpg  
  inflating: dataset/1_airliner.jpg  
  inflating: dataset/20_Japanese spaniel.jpg  
  inflating: dataset/21_Japanese spaniel.jpg  
  inflating: dataset/22_Japanese spaniel.jpg  
  inflating: dataset/23_Japanese spaniel.jpg  
  inflating: dataset/24_Japanese spaniel.jpg  
  inflating: dataset/25_Japanese spaniel.jpg  
  inflating: dataset/26_kuvasz.jpg   
  inflating: dataset/27_kuvasz.jpg   
  inflating: dataset/28_kuvasz.jpg   
  inflating: dataset/29_kuvasz.jpg   
  inflating: dataset/2_airliner.jpg 

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [4]:
def get_acc(model):
  folder_dir = "/content/dataset"
  top_5_acc_num = 0 
  top_1_acc_num = 0 
  num_of_pics = 0
  errors = 0
  for image in os.listdir(folder_dir):
        num_of_pics = num_of_pics + 1
        if (image.endswith(".jpg")):
            img = Image.open(folder_dir+'/'+image)
            list_of_top_5_preds = model(img)
            print(image)
            print(list_of_top_5_preds)
            real_name = (image[image.find('_')+1:image.find('.')])
            if real_name in list_of_top_5_preds:
                top_5_acc_num = top_5_acc_num + 1
            if real_name==list_of_top_5_preds[0]:
                top_1_acc_num = top_1_acc_num + 1
  return top_5_acc_num/num_of_pics , top_1_acc_num/num_of_pics

### AlexNet

In [17]:
from torchvision.models import alexnet, AlexNet_Weights

In [18]:
weights_alexnet = AlexNet_Weights.IMAGENET1K_V1
model = alexnet(weights_alexnet)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [19]:
def alexnet(image): 
  image = preprocess(image)         
  img = image.unsqueeze(0) 
  if torch.cuda.is_available():
    img = img.to('cuda')
    model.to('cuda')
  prediction = model(img).squeeze(0).softmax(0)
  n = 5
  cat_names = []
  class_id = torch.topk(prediction.flatten(), n).indices
  for i in range(n):
    score = prediction[class_id[i]].item()
    category_name = weights.meta["categories"][class_id[i]]
    print(f"{category_name}: {100 * score:.1f}%")
    cat_names.append(category_name)
  return cat_names

In [20]:
%%time
t_5_acc, t_1_acc = get_acc(alexnet)
print(f'Top 5 accuracy:{np.round(t_5_acc,3)}, Top 1 accuracy:{np.round(t_1_acc,3)}')

bookshop: 28.2%
comic book: 26.9%
library: 17.1%
toyshop: 6.8%
confectionery: 4.9%
36_comic book.jpg
['bookshop', 'comic book', 'library', 'toyshop', 'confectionery']
airliner: 97.1%
wing: 0.9%
airship: 0.9%
warplane: 0.7%
projectile: 0.1%
6_airliner.jpg
['airliner', 'wing', 'airship', 'warplane', 'projectile']
Japanese spaniel: 80.1%
papillon: 19.7%
Pomeranian: 0.1%
Pekinese: 0.0%
Shih-Tzu: 0.0%
19_Japanese spaniel.jpg
['Japanese spaniel', 'papillon', 'Pomeranian', 'Pekinese', 'Shih-Tzu']
goose: 52.8%
albatross: 38.9%
ptarmigan: 3.9%
spoonbill: 2.5%
white stork: 0.9%
18_goose.jpg
['goose', 'albatross', 'ptarmigan', 'spoonbill', 'white stork']
kuvasz: 99.3%
white wolf: 0.4%
Great Pyrenees: 0.2%
ram: 0.0%
golden retriever: 0.0%
31_kuvasz.jpg
['kuvasz', 'white wolf', 'Great Pyrenees', 'ram', 'golden retriever']
starfish: 99.8%
walking stick: 0.1%
knot: 0.0%
nail: 0.0%
scorpion: 0.0%
43_starfish.jpg
['starfish', 'walking stick', 'knot', 'nail', 'scorpion']
starfish: 80.4%
matchstick: 19.0

In [22]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla T4
Memory Usage:
Allocated: 0.2 GB
Cached:    0.7 GB


In [23]:
print(round(torch.cuda.max_memory_allocated()/1024**3,1), 'GB')
if device == 'cuda':
    torch.cuda.reset_max_memory_allocated()

0.6 GB


### VGG19

In [11]:
from torchvision.models import vgg19
from torchvision.models.vgg import VGG19_Weights

In [12]:
weights_vgg19 = VGG19_Weights.IMAGENET1K_V1
model = vgg19(weights=weights_vgg19)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [13]:
def vgg19(image): 
  image = preprocess(image)          
  img = image.unsqueeze(0)   
  if torch.cuda.is_available():
    img = img.to('cuda')
    model.to('cuda')
  prediction = model(img).squeeze(0).softmax(0)
  n = 5
  cat_names = []
  class_id = torch.topk(prediction.flatten(), n).indices
  for i in range(n):
    score = prediction[class_id[i]].item()
    category_name = weights.meta["categories"][class_id[i]]
    print(f"{category_name}: {100 * score:.1f}%")
    cat_names.append(category_name)
  return cat_names

In [14]:
%%time
t_5_acc, t_1_acc = get_acc(vgg19)
print(f'Top 5 accuracy:{np.round(t_5_acc,3)}, Top 1 accuracy:{np.round(t_1_acc,3)}')

comic book: 93.0%
bookshop: 6.3%
library: 0.3%
packet: 0.1%
binder: 0.1%
36_comic book.jpg
['comic book', 'bookshop', 'library', 'packet', 'binder']
airliner: 95.7%
warplane: 1.9%
wing: 1.9%
missile: 0.2%
projectile: 0.2%
6_airliner.jpg
['airliner', 'warplane', 'wing', 'missile', 'projectile']
Japanese spaniel: 99.0%
Pekinese: 0.9%
titi: 0.0%
papillon: 0.0%
marmoset: 0.0%
19_Japanese spaniel.jpg
['Japanese spaniel', 'Pekinese', 'titi', 'papillon', 'marmoset']
goose: 97.6%
crane bird: 1.8%
hen: 0.3%
albatross: 0.1%
drake: 0.1%
18_goose.jpg
['goose', 'crane bird', 'hen', 'albatross', 'drake']
kuvasz: 99.9%
Great Pyrenees: 0.1%
white wolf: 0.0%
komondor: 0.0%
Samoyed: 0.0%
31_kuvasz.jpg
['kuvasz', 'Great Pyrenees', 'white wolf', 'komondor', 'Samoyed']
starfish: 100.0%
sea urchin: 0.0%
window screen: 0.0%
matchstick: 0.0%
sea slug: 0.0%
43_starfish.jpg
['starfish', 'sea urchin', 'window screen', 'matchstick', 'sea slug']
starfish: 97.4%
knot: 2.3%
hammerhead: 0.0%
sea cucumber: 0.0%
scorpi

In [15]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla T4
Memory Usage:
Allocated: 0.5 GB
Cached:    0.7 GB


In [16]:
print(round(torch.cuda.max_memory_allocated()/1024**3,1), 'GB')
if device == 'cuda':
    torch.cuda.reset_max_memory_allocated()

0.6 GB


### ResNet50

In [5]:
from torchvision.models import resnet50, ResNet50_Weights

In [6]:
weights = ResNet50_Weights.IMAGENET1K_V1
model = resnet50(weights=weights)
model.eval()
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25]),
])

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [7]:
def resnet(img):
  image = preprocess(img).unsqueeze(0)  

  if torch.cuda.is_available():
    image = image.to('cuda')
    model.to('cuda')
        
  prediction = model(image).squeeze(0).softmax(0)
  n = 5
  class_id = torch.topk(prediction.flatten(), n).indices
  cat_names = []
  for i in range(n):
    score = prediction[class_id[i]].item()
    category_name = weights.meta["categories"][class_id[i]]
    print(f"{category_name}: {100 * score:.1f}%")
    cat_names.append(category_name)
  return cat_names

In [8]:
%%time
t_5_acc, t_1_acc = get_acc(resnet)
print(f'Top 5 accuracy:{np.round(t_5_acc,3)}, Top 1 accuracy:{np.round(t_1_acc,3)}')

comic book: 88.2%
library: 6.8%
bookshop: 4.5%
toyshop: 0.3%
binder: 0.1%
36_comic book.jpg
['comic book', 'library', 'bookshop', 'toyshop', 'binder']
airliner: 92.1%
wing: 5.2%
warplane: 1.7%
projectile: 0.5%
missile: 0.4%
6_airliner.jpg
['airliner', 'wing', 'warplane', 'projectile', 'missile']
Japanese spaniel: 98.8%
papillon: 0.5%
Pekinese: 0.2%
Shih-Tzu: 0.1%
Chihuahua: 0.0%
19_Japanese spaniel.jpg
['Japanese spaniel', 'papillon', 'Pekinese', 'Shih-Tzu', 'Chihuahua']
goose: 99.5%
black swan: 0.2%
crane bird: 0.1%
hen: 0.1%
albatross: 0.0%
18_goose.jpg
['goose', 'black swan', 'crane bird', 'hen', 'albatross']
kuvasz: 99.8%
Great Pyrenees: 0.1%
ram: 0.0%
Samoyed: 0.0%
white wolf: 0.0%
31_kuvasz.jpg
['kuvasz', 'Great Pyrenees', 'ram', 'Samoyed', 'white wolf']
starfish: 100.0%
sea urchin: 0.0%
coral reef: 0.0%
stinkhorn: 0.0%
nail: 0.0%
43_starfish.jpg
['starfish', 'sea urchin', 'coral reef', 'stinkhorn', 'nail']
starfish: 97.2%
matchstick: 0.2%
drumstick: 0.1%
stinkhorn: 0.1%
slug: 0.

In [24]:
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Tesla T4
Memory Usage:
Allocated: 0.2 GB
Cached:    0.7 GB


In [25]:
print(round(torch.cuda.max_memory_allocated()/1024**3,1), 'GB')
if device == 'cuda':
    torch.cuda.reset_max_memory_allocated()

0.6 GB
